In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, ParameterGrid, cross_validate, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import optuna
from sklearn.metrics import make_scorer

In [14]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.isnull().sum()

id        0
cat0      0
cat1      0
cat2      0
cat3      0
cat4      0
cat5      0
cat6      0
cat7      0
cat8      0
cat9      0
cont0     0
cont1     0
cont2     0
cont3     0
cont4     0
cont5     0
cont6     0
cont7     0
cont8     0
cont9     0
cont10    0
cont11    0
cont12    0
cont13    0
target    0
dtype: int64

In [15]:
df_train.dtypes

id          int64
cat0       object
cat1       object
cat2       object
cat3       object
cat4       object
cat5       object
cat6       object
cat7       object
cat8       object
cat9       object
cont0     float64
cont1     float64
cont2     float64
cont3     float64
cont4     float64
cont5     float64
cont6     float64
cont7     float64
cont8     float64
cont9     float64
cont10    float64
cont11    float64
cont12    float64
cont13    float64
target    float64
dtype: object

In [16]:
def Encode(df):
    le = LabelEncoder() 
    for column in df.select_dtypes('object'):
        le.fit(df[column])
        df[column]=le.transform(df[column])
    
    scaler = StandardScaler()
    for column in df.select_dtypes('float64'):
        scaler.fit(df[column].values.reshape(-1,1))
        df[column] = scaler.transform(df[column].values.reshape(-1,1))
         
    return df

In [21]:
#ハイパーパラメーターチューニング＋交差検証

def objective(trial):
    X_train = df_train.drop('target', axis=1)
    y_train = df_train['target']
    
    X_train = Encode(X_train)
    
    xgb_params = dict(
        max_depth=trial.suggest_int("max_depth", 5, 10),
        learning_rate=trial.suggest_float("learning_rate", 1e-2, 1e-1, log=True),
        n_estimators=500
#         min_child_weight=trial.suggest_int("min_child_weight", 1, 10),
#         colsample_bytree=trial.suggest_float("colsample_bytree", 0.2, 1.0),
#         subsample=trial.suggest_float("subsample", 0.2, 1.0),
#         reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e2, log=True),
#         reg_lambda=trial.suggest_float("reg_lambda", 1e-4, 1e2, log=True),
    )
    
    clf = XGBRegressor(**xgb_params)
    kf = KFold(n_splits=5, random_state=0, shuffle=True)
   
    def RMSE(y_pred, y_true):
        rmse = np.sqrt(mean_squared_error(y_pred,y_true))
        
        return rmse 
        
    some_funcs ={
        'RMSE':make_scorer(RMSE)
    }
    
    scores = cross_validate(clf, X=X_train, y=y_train, cv=kf, scoring=some_funcs)
    
    return scores['test_RMSE'].mean()

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)
xgb_params = study.best_params

# 実行結果表示
print('最終トライアル回数:{}'.format(len(study.trials)))
print('ベストトライアル:')
trial = study.best_trial
print('値:{}'.format(trial.value))
print('パラメータ:')
for key, value in trial.params.items():
    print('{}:{}'.format(key, value))

[I 2021-02-28 07:12:15,990] A new study created in memory with name: no-name-88baaf3e-e4df-4908-bf1e-4bcf7f0b41df
[I 2021-02-28 07:23:37,311] Trial 0 finished with value: 0.8451696003569937 and parameters: {'max_depth': 5, 'learning_rate': 0.0412346755521118}. Best is trial 0 with value: 0.8451696003569937.
[I 2021-02-28 07:37:31,583] Trial 1 finished with value: 0.8457216048685808 and parameters: {'max_depth': 6, 'learning_rate': 0.0269893000009116}. Best is trial 0 with value: 0.8451696003569937.
[I 2021-02-28 07:48:46,341] Trial 2 finished with value: 0.8447281195337972 and parameters: {'max_depth': 5, 'learning_rate': 0.08528713029565459}. Best is trial 2 with value: 0.8447281195337972.
[I 2021-02-28 08:07:38,283] Trial 3 finished with value: 0.8459860747668273 and parameters: {'max_depth': 8, 'learning_rate': 0.02264942072724693}. Best is trial 2 with value: 0.8447281195337972.
[I 2021-02-28 08:19:01,550] Trial 4 finished with value: 0.8453431029400417 and parameters: {'max_depth'

最終トライアル回数:5
ベストトライアル:
値:0.8447281195337972
パラメータ:
max_depth:5
learning_rate:0.08528713029565459


In [28]:
X_train = df_train.drop('target', axis=1)
y_train = df_train['target']
X_train = Encode(X_train)
X_test = Encode(df_test)

clf = XGBRegressor(**xgb_params)
clf.fit(X_train, y_train)
pred_test = clf.predict(X_test)
df_sample = pd.read_csv('sample_submission.csv')
df_sample['target'] = pred_test
df_sample.to_csv('submmit.csv', index=False)